In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
#import data
data = pd.read_csv('/kaggle/input/questions.csv')

/kaggle/input/questions.csv


In [2]:
#check slice of data
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
# distribution of data for different class
data.is_duplicate.value_counts()

0    255045
1    149306
Name: is_duplicate, dtype: int64

In [4]:
print('length of data: ', len(data))
print('shape of data: ', data.shape)
#no of entries in data is 405k which is quiet high 
#so we can perform analysis on subset of data to save excecution time.

length of data:  404351
shape of data:  (404351, 6)


In [5]:
# taking subset of data 
df = data[:50000] 


In [6]:
# checking Nan values if any
df.isna().sum()
#we dont have any Nan values in subset of data

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
#printing few question pairs in the datset
for i in range(0,20):
    print(df.question1[i])
    print(df.question2[i])
    print()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?

Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?

Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?

Should I buy tiago?
What keeps childern active and far from phone and video games?

How can I be a good geologist?
What should I do to be a great geologist?

When do you use シ instea

In [8]:
from string import punctuation
import scipy #library for scientific calculations
import datetime
import nltk
from sklearn import re
from sklearn import pipeline
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

In [9]:
# Cleaning text in question

# This list is so extensive that we can look back at it as a reference when 
# needed for other regular expression related work.

SPECIAL_TOKENS = {'non-ascii': 'non_ascii_word'}

def normalized_text(text, stem_words=True):
    def pad_str(s):
        return ' '+s+' '
    
    if pd.isnull(text):  #If null
        return ''

    # Empty question
    
    if type(text) != str or text=='':  #if text type is not string
        return ''

    # Clean the text
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE) # replace whats by what is and ignore case
    text = re.sub("\'ve", " have ", text) # replace 've by have
    text = re.sub("can't", "can not", text) # replace can't by can not
    text = re.sub("n't", " not ", text) # replace n't by not
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE) # replace i'm by i am and ignore case
    text = re.sub("\'re", " are ", text) # replace 're by are
    text = re.sub("\'d", " would ", text) # replace 'd by would
    text = re.sub("\'ll", " will ", text) # replace 'll by will
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE) # replace e.g. by eg and ignore case
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE) # replace b.g. by bg and ignore case
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text) 
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
    # add padding to punctuations and special chars, we still need them later
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) 
    
    # Indian Currency
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)
    
    # cleaning text rules from : https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" banglore ", " Banglore ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    
    # replace the float numbers with a random number
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
  
    #Removing Punctuations
    text = [word for word in text if word not in punctuation]
    text = ''.join(text)
    text = text.lower()
       
    # Return a list of words
    return text

In [10]:
#applying text cleaning function to question text
df['question1'] = df['question1'].apply(normalized_text)
df['question2'] = df['question2'].apply(normalized_text)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#checking the cleaned text to see changes made
for i in range(0,20):
    print(df.question1[i])
    print(df.question2[i])
    print()

what is the step by step guide to invest in share market in india
what is the step by step guide to invest in share market

what is the story of kohinoor kohinoor diamond
what would happen if the indian government stole the kohinoor kohinoor diamond back

how can i increase the speed of my internet connection while using a vpn
how can internet speed be increased by hacking through dns

why am i mentally very lonely how can i solve it
find the remainder when math2324math is divided by 2423

which one dissolve in water quikly sugar salt methane and carbon di oxide
which fish would survive in salt water

astrology i am a capricorn sun cap moon and cap risingwhat does that say about me
i am a triple capricorn sun moon and ascendant in capricorn what does this say about me

should i buy tiago
what keeps childern active and far from phone and video games

how can i be a good geologist
what should i do to be a great geologist

when do you use  nonasciiword  instead of  nonasciiword 
when do y

In [12]:
#data head
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


* 1. * *** Bag of Words + XGBOOST**


In [13]:
#using word vector of word_count and frequency withpout capturing the meaning of word
#r'\w{1,}' indiactes 1 or more word

CV = CountVectorizer(analyzer='word', stop_words='english', token_pattern=r'\w{1,}')
q1_trans = CV.fit_transform(df['question1'].values)
q2_trans = CV.fit_transform(df['question2'].values)
    

In [14]:
#scipy.sparse.hstack will stack sparse matrix columnwise, and stacking them side by side

X = scipy.sparse.hstack((q1_trans, q2_trans))
y = df.is_duplicate.values    

In [15]:
#splitting traing set and test set for training and validating model for classification
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

In [16]:
#gradient Boosting Model used
#start time
st = datetime.datetime.now()

classifier1 = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
#fitting the model
print(classifier1.fit(X_train, y_train))
#predicting if pair is duplicate or not
prediction_CV = classifier1.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, prediction_CV))
print("Accuracy score: \n", accuracy_score(y_test, prediction_CV))
print("Classification report:\n", classification_report(y_test, prediction_CV))
print("F1 Score:\n ",f1_score(y_test, prediction_CV))

et = datetime.datetime.now()
print("Code run-time: ", et-st)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)
Confusion Matrix:
 [[8545  833]
 [3290 2332]]
Accuracy score: 
 0.7251333333333333
Classification report:
               precision    recall  f1-score   support

           0       0.72      0.91      0.81      9378
           1       0.74      0.41      0.53      5622

    accuracy                           0.73     15000
   macro avg       0.73      0.66      0.67     15000
weighted avg       0.73      0.73      0.70     15000

F1 Score:
  0.530784112894048
Code run-time:  0:02:16.041230


*** ****TF-IDF (word level) + XGBOOST
**

In [17]:
#TF-IDF vectorizer 
#5000 features were used for tfidf vectorizer
#r'\w{1,}'  indicates more than 1 word

tfidf = TfidfVectorizer(analyzer='word', max_features=5000, token_pattern=r'\w{1,}')

q1word_trans = tfidf.fit_transform(df['question1'].values)
q2word_trans = tfidf.fit_transform(df['question2'].values)

X = scipy.sparse.hstack((q1word_trans,q2word_trans))
y = df.is_duplicate.values


In [18]:
# train-test-split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [19]:
# Xg Boost classifier for word level vectorizer

st = datetime.datetime.now()

classifier2 = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
#fitting the model with traing data
print(classifier2.fit(X_train, y_train))

#predicting the test data
prediction_tfidf = classifier2.predict(X_test)

#Performance evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction_tfidf))
print("Accuracy score: \n", accuracy_score(y_test, prediction_tfidf))
print("Classification report:\n", classification_report(y_test, prediction_tfidf))
print("F1 Score:\n ",f1_score(y_test, prediction_tfidf))

et = datetime.datetime.now()
print("Code run-time: ", et-st)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)
Confusion Matrix:
 [[8932 1400]
 [2835 3333]]
Accuracy score: 
 0.7433333333333333
Classification report:
               precision    recall  f1-score   support

           0       0.76      0.86      0.81     10332
           1       0.70      0.54      0.61      6168

    accuracy                           0.74     16500
   macro avg       0.73      0.70      0.71     16500
weighted avg       0.74      0.74      0.73     16500

F1 Score:
  0.6115035317860747
Code run-time:  0:01:36.557736


* ******ngram level TFIDF + XGBOOST

In [20]:
#TF-IDF ngram level vectorizer 
#5000 features were used for tfidf vectorizer
#r'\w{1,}'  indicates more than 1 word
#ngram_range = (1,3) means 2 and 3 features are used

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1,3), max_features=5000, token_pattern=r'\w{1,}')

q1ngram_trans = tfidf.fit_transform(df['question1'].values)
q2ngram_trans = tfidf.fit_transform(df['question2'].values)

X = scipy.sparse.hstack((q1ngram_trans,q2ngram_trans))
y = df.is_duplicate.values

In [21]:
# train-test-split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

In [22]:
# Xg Boost classifier for ngram_range=(1,3) level vectorizer

st = datetime.datetime.now()

classifier3 = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
#fitting the model with traing data
print(classifier3.fit(X_train, y_train))

#predicting the test data
prediction_tfidf = classifier3.predict(X_test)

#Performance evaluation
print("ngram_range Confusion Matrix:\n", confusion_matrix(y_test, prediction_tfidf))
print("ngram_range Accuracy score: \n", accuracy_score(y_test, prediction_tfidf))
print("ngram_range Classification report:\n", classification_report(y_test, prediction_tfidf))
print("ngram_range F1 Score:\n ",f1_score(y_test, prediction_tfidf))

et = datetime.datetime.now()
print("Code run-time: ", et-st)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)
ngram_range Confusion Matrix:
 [[8095 1283]
 [2662 2960]]
ngram_range Accuracy score: 
 0.737
ngram_range Classification report:
               precision    recall  f1-score   support

           0       0.75      0.86      0.80      9378
           1       0.70      0.53      0.60      5622

    accuracy                           0.74     15000
   macro avg       0.73      0.69      0.70     15000
weighted avg       0.73      0.74      0.73     15000

ngram_range F1 Score:
  0.6001013684744045
Code run-time:  0:02

In [23]:
#TF-IDF ngram level vectorizer 
#5000 features were used for tfidf vectorizer
#r'\w{1,}'  indicates more than 1 word
#ngram_range = (2,3) means 2 and 3 features are used

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(2,3), max_features=5000, token_pattern=r'\w{1,}')

q1ngram_trans = tfidf.fit_transform(df['question1'].values)
q2ngram_trans = tfidf.fit_transform(df['question2'].values)

X = scipy.sparse.hstack((q1ngram_trans,q2ngram_trans))
y = df.is_duplicate.values

In [24]:
# train-test-split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

In [25]:
# Xg Boost classifier for ngram_range=(1,3) level vectorizer

st = datetime.datetime.now()

classifier4 = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
#fitting the model with traing data
print(classifier4.fit(X_train, y_train))

#predicting the test data
prediction_tfidf = classifier4.predict(X_test)

#Performance evaluation
print("ngram_range(2,3) Confusion Matrix:\n", confusion_matrix(y_test, prediction_tfidf))
print("ngram_range(2,3) Accuracy score: \n", accuracy_score(y_test, prediction_tfidf))
print("ngram_range(2,3) Classification report:\n", classification_report(y_test, prediction_tfidf))
print("ngram_range(2,3) F1 Score:\n ",f1_score(y_test, prediction_tfidf))

et = datetime.datetime.now()
print("Code run-time: ", et-st)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)
ngram_range(2,3) Confusion Matrix:
 [[8282 1096]
 [3322 2300]]
ngram_range(2,3) Accuracy score: 
 0.7054666666666667
ngram_range(2,3) Classification report:
               precision    recall  f1-score   support

           0       0.71      0.88      0.79      9378
           1       0.68      0.41      0.51      5622

    accuracy                           0.71     15000
   macro avg       0.70      0.65      0.65     15000
weighted avg       0.70      0.71      0.68     15000

ngram_range(2,3) F1 Score:
  0.5100

*** Char level TFIDF + XGBOOST****

In [26]:
#TF-IDF ngram level vectorizer 
#5000 features were used for tfidf vectorizer
#r'\w{1,}'  indicates more than 1 word
#ngram_range = (1,3) means 2 and 3 features are used
#char level analyzer is used 

tfidf = TfidfVectorizer(analyzer='char',ngram_range=(1,3), max_features=5000, token_pattern=r'\w{1,}')

q1char_trans = tfidf.fit_transform(df['question1'].values)
q2char_trans = tfidf.fit_transform(df['question2'].values)

X = scipy.sparse.hstack((q1char_trans,q2char_trans))
y = df.is_duplicate.values

In [27]:
# train-test-split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

In [28]:
# Xg Boost classifier for char level vectorizer

st = datetime.datetime.now()

classifier5 = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
#fitting the model with traing data
print(classifier5.fit(X_train, y_train))

#predicting the test data
prediction_tfidf = classifier5.predict(X_test)

#Performance evaluation
print("char level Confusion Matrix:\n", confusion_matrix(y_test, prediction_tfidf))
print("char level Accuracy score: \n", accuracy_score(y_test, prediction_tfidf))
print("char level Classification report:\n", classification_report(y_test, prediction_tfidf))
print("char level F1 Score:\n ",f1_score(y_test, prediction_tfidf))

et = datetime.datetime.now()
print("Code run-time: ", et-st)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)
char level Confusion Matrix:
 [[8338 1040]
 [2600 3022]]
char level Accuracy score: 
 0.7573333333333333
char level Classification report:
               precision    recall  f1-score   support

           0       0.76      0.89      0.82      9378
           1       0.74      0.54      0.62      5622

    accuracy                           0.76     15000
   macro avg       0.75      0.71      0.72     15000
weighted avg       0.76      0.76      0.75     15000

char level F1 Score:
  0.624122263527468
Code run-tim

**Conclusion**

In [29]:
### Challenge was to create the model which can classify between similar question asked on quora##

### text processing used is so extensive that we can use it as a reference when 
# needed for other regular expression related work.###

### Initially Bag of Words model is used for vectorization with XGBOOST for classification

### TFIDF vectorizer at word level, n-Gram level, Char level is used.

### Without sentiment capturing, best accuracy of 75.73% is achieved using 
###extreme gradient boosting on character level TF-IDF....